# Robot Trading

## Parte 1: Configuración del ambiente

***Vamos a realizar la importación de las bibliotecas y las variables globales que necesitaremos para trabajar en el proyecto***

In [51]:
import pandas as pd

import numpy as np

import yfinance as yf

%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('figure', figsize = (19.2, 10.8))

from bs4 import BeautifulSoup; import requests

global algoritmo_decision, df_bitcoin, df_bitcoin_limpio, media_bitcoin, precio_actual, tendencia

## Parte 2: Obtención de datos

***En esta sección crearemos las dos funciones necesarias para obtener el DataFrame de los valores históricos del Bitcoin, el precio actual y el indicador de tendencia***

### Definición de funciones

In [52]:
def importar_base_bitcoin():
    
    global df_bitcoin
    df_bitcoin = yf.Ticker('BTC-USD').history(period = '7d', interval = '5m')
#Función de obtención de los datos históricos del valor del Bitcoin (BTC-USD)
# en un periodo de una semana con intervalos de 5 minutos

def extraer_tendencias():
    
    global precio_actual, tendencia
    
    url = 'https://coinmarketcap.com/'
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/113.0.0.0 Safari/537.36'}
    response = requests.get(url, headers = headers)
    soup = (BeautifulSoup(response.content, 'html.parser')).tbody
    filas = soup.find_all('p', {'class' : 'sc-4984dd93-0 iqdbQL coin-item-symbol'} )
    indice_fila = 0

    for row in filas:
        
        if row.getText() == 'BTC':
            
            break
        
        else:
            
            indice_fila += 1
    #Con este contador ubicamos la fila en la que se encuentra el Bitcoin, esto está pensado para casos en los que ya no se
    # encuentre en primera posición, pero también es fácilmente modificable para obtener los valores de otras criptomonedas

    precio_actual = float(soup.find_all('tr')[indice_fila].find_all('td')[3].getText().replace('$', '').replace(',', ''))
    tendencia = ''
    
    if soup.find_all('tr')[indice_fila].find_all('td')[4].span.span['class'][0] == 'icon-Caret-up':
        
        tendencia = 'Alta'
        
    else:
        
        tendencia = 'Baja'
        
    print('El precio actual y la tendencia son:', precio_actual, tendencia)
#Función que obtiene el valor del precio actual del Bitcoin y el indicador de tendencia

# Parte 3: Limpieza de datos

In [53]:
def limpieza_datos():
    
    global df_bitcoin, df_bitcoin_limpio, media_bitcoin
    df_bitcoin_limpio = df_bitcoin.copy()
    df_bitcoin_limpio = df_bitcoin_limpio[~df_bitcoin.index.duplicated()]
    #Copiamos el DataFrame original para trabajar sobre la copia y eliminamos los índices duplicados

    df_bitcoin_limpio.dropna(subset = ['Close'], inplace=True)
    #Eliminamos todos los valores 'Nan' de la columna 'Close' del DataFrame
    
    df_bitcoin_limpio = df_bitcoin_limpio[df_bitcoin_limpio['Volume'] > 0]
    #Eliminamos todos los valores iguales o menores a cero de la columna 'Volume' del DataFrame
    
    Valor = df_bitcoin_limpio['Close']
    Q1 = Valor.quantile(.25)
    Q3 = Valor.quantile(.75)
    IIQ = Q3 - Q1
    limite_inferior = Q1 - (1.5 * IIQ)
    limite_superior = Q3 + (1.5 * IIQ)
    #Calculamos los valores de nuestra gráfica boxplot con los que filtraremos los datos

    filtro_outliers = (Valor >= limite_inferior) & (Valor <= limite_superior)
    df_bitcoin_limpio = df_bitcoin_limpio[filtro_outliers]
    #Con estas dos líneas de código eliminamos los outliers en caso de existir

    filtro_q1_q3 = (Valor >= Q1) & (Valor <= Q3)
    df_bitcoin_limpio = df_bitcoin_limpio[filtro_q1_q3]
    #Con estas dos líneas de código seleccionamos solo los valores entre el Q1 y Q3

    df_bitcoin_limpio.reset_index(inplace = True)
    #Reasignamos el index después de haber hecho el filtro a los datos
    
    media_bitcoin = round(df_bitcoin_limpio['Close'].mean(), 2)
    #Calculamos la media de los valores de la columna 'Close' del DataFrame y redondeamos a 2 decimales
    
    print('El precio promedio del bitcoin es:', media_bitcoin)

In [54]:
importar_base_bitcoin()
extraer_tendencias()
limpieza_datos()

El precio actual y la tendencia son: 26804.05 Alta
El precio promedio del bitcoin es: 27334.1
